# 1. lstm
* 用pytorch做listm模型
* 将长度变化的句子输入lism 得到隐藏层和输出成，当做编码的结果。
* 如果效果不好的话可以试一下，crf?attention?
        
# 2.  现在的问题。

* 预测的时候还要重新计算词向量
* 加入embeding层
        

In [1]:
import jieba 
import numpy as np
import sklearn
import torch
from torch import nn
from sklearn.metrics import f1_score
import torch.nn.functional as F
from batch_size import datasets
from torch.utils.data import Dataset, DataLoader
from preprocess import pre_train
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd
import config as c

In [2]:
load_word=pre_train()
data=load_word.load_word()

## 1.1 word_embeding
如果需要自己训练embeding
如果内存真的放不下，最好用embeding

In [3]:
def is_chinese(uchar):         
    if '\u4e00' <= uchar<='\u9fff':
        return True
    else:
        return False

def jiebaclearText(data,stopwords_path=r'../data/stop_words.txt'):
    '''保留所有的汉字。
    '''
    for sents in data['q1']:
        '''句子'''
        for w in sents:
            '''一个单词'''
            if not is_chinese(w):
                sents.remove(w)
    for sents in data['q2']:
        '''句子'''
        for w in sents:
            '''一个单词'''
            if not is_chinese(w):
                sents.remove(w)
                
    return data
data=jiebaclearText(data)

In [ ]:
data['q1'][0]

### 1.2获得词汇
vocab

In [4]:

def get_vocab():
    vocab=set()
    for i in data['q1']:
        for j in i:
            vocab.add(j)
    for i in data['q2']:
        for j in i:
            vocab.add(j)
    return list(vocab)
vocab=get_vocab()

vocab to index

In [5]:
word2index={i:index for index,i in enumerate(vocab)}

index2words

In [ ]:
index2word={index:i for index,i in enumerate(vocab)}

## 2.构建Ngarma模型-----不需要
     输入词汇的个数，编码后的维度，以及每个x有几个单词
     不需要，我可以直接把embeding和lstm合并

## 1.3. 数据获取
* 获得所有word2vec的矢量
* 首先知道需要的矩阵大小初始化一个zeros

In [6]:
'花呗' in word2index

False

* 给我一个索引我返回该索引对应的两个句子叠加后的索引和目标函数

In [6]:
def get_data_lstm(idx):
    '''
    就是把数据传入,batch_size=1
    seq_len ,word_dim
    '''
    idx=idx[0]
    words=data['q1'][idx]+data['q2'][idx]
    batch=np.zeros((len(words)))
    for i,w in enumerate(words):
        if w in word2index:
            batch[i]=word2index[w]
        else:
            batch[i]=-1
    target=np.array([data['target'][idx]])    
    return batch,target 
x,y=get_data_lstm([0])

In [ ]:
x

### 2. lstm得到隐藏层

* 我希望把每个句子（seq_len）内单词对应的矢量按照顺序输入之后得到一个维度相同的隐藏层。
* 有两种情况，
    * 我把两个句子合并输入，直接得到x,y
    * 我把两个句子输入之后得到输出，然后在把输出合并，得到x,y

## 2.1 第一种

In [7]:
class lstm_embedding(nn.Module):
    def __init__(self,vocab_size,word_dim,hidden_dim,target_size,batch_size=1,):
        '''
        输入一个长句子，包含很多词向量seq_len。
        得到hidden,然后用hidden做x预测y
        
        '''
        super(lstm_embedding,self).__init__()
        self.embd=nn.Embedding(vocab_size,word_dim)
        self.lstm=nn.LSTM(word_dim,hidden_dim)
        self.hidden2target=nn.Linear(hidden_dim,target_size)
        
        self.batch_size=batch_size
        self.hidden_dim=hidden_dim
        self.hidden=self.init_hidden()
        self.word_dim=word_dim
        self.vocab_size=vocab_size
        
    def init_hidden(self):
        '''两个句子之间没有联系，所有我每个句子都要随机初始化
        # 各个维度的含义是 (num_layers, minibatch_size, hidden_dim)
        '''     
        return (torch.autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                torch.autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))  
    
    def forward(self,x):
        #seq,batch,hidden
        x=self.embd(x)
        output,self.hidden=self.lstm(x.view(len(x),self.batch_size,self.word_dim),self.hidden)
        out=self.hidden2target(self.hidden[0].view(-1,self.hidden_dim))
        out=F.log_softmax(out,dim=1)
        return out

损失函数

In [ ]:
c.word_dim

In [ ]:
len(vocab)

In [10]:
hidden_dim=128
target_size=2
model = lstm_embedding(len(vocab),c.word_dim, hidden_dim,target_size)
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.5e-1)

训练

In [ ]:
epochs=1
for epoch in range(epochs):
    '''
    重新采样
    '''
    model.train()
    dataset=datasets(sample=True)
    datas=DataLoader(dataset,batch_size=1,shuffle=True)
    total_loss=torch.tensor(0.0)
    for i,idx in enumerate(datas):
        '''
        把两个序列链接起来。
        '''
        train_x,train_y=get_data_lstm(idx.numpy())
        inputs=Variable(torch.LongTensor(train_x))
        labels=Variable(torch.tensor(train_y).long())
        #moxing
        model.zero_grad()
        model.hidden=model.init_hidden()
        # Step 3. 前向传播
        tag_scores = model(inputs)
        # Step 4. 计算损失和梯度值 通过调用 optimizer.step() 来更新梯度
        loss = loss_function(tag_scores, labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        if i%200==0:
            print(epoch,i,total_loss/200)
            total_loss=torch.tensor(0.0)

#### 评估

In [11]:
model.load_state_dict(torch.load('../model/lstm_v2_100_state_ditc.pkl'))

In [21]:
y_pred0=[]
y=[]
model.eval()
with torch.no_grad():
    for i in range(len(data['index'])):
        train_x,train_y=get_data_lstm([i])
        inputs=Variable(torch.LongTensor(train_x))
        labels=Variable(torch.tensor(train_y).long())
        y_pred0.append(model(inputs))
        y.append(labels)   
        


In [24]:
y_pred=[i.argmax() for i in y_pred0]

print('f1_score',f1_score(y,y_pred))

f1_score 0.04639271104499815


In [25]:
y_pred=[i.argmin() for i in y_pred0]

print('f1_score',f1_score(y,y_pred))

f1_score 0.30736468500443653


### 保存模型

In [ ]:
torch.save(model,'../model/lstm_v2-100.pkl')

In [12]:
model=torch.load('./model/lstm_v2-100.pkl')

In [ ]:
type(model1)

In [ ]:
len(vocab)

In [ ]:
torch.save(model.state_dict(),'../model/lstm_v2_100_state_ditc.pkl')

# 第二种
 * 把两个的输出链接起来然后再预测
 * 还没有修改完毕

In [ ]:
class lstm_embedding_cat(nn.Module):
    '''
    把两个句子的隐藏变量结合之后用线性函数预测。
    '''
    def __init__(self,word_dim,hidden_dim,target_size,batch_size=1,):
        '''
        输入一个长句子，包含很多词向量seq_len。
        得到hidden,然后用hidden做x预测y        
        '''
        super(lstm_embedding_cat,self).__init__()
        #参数
        self.batch_size=batch_size
        self.hidden_dim=hidden_dim
        self.word_dim=word_dim
        
        #第一个句子        
        self.lstm_0=nn.LSTM(word_dim,hidden_dim)        
        #第二个句子
        self.lstm_1=nn.LSTM(word_dim,hidden_dim)       
        
        #合并线性
        self.linear=nn.Linear(2*hidden_dim,2)
        
        #初始化隐藏过程
        self.hidden_0=self.init_hidden()        
        self.hidden_1=self.init_hidden()        
    def init_hidden(self):
        '''两个句子之间没有联系，所有我每个句子都要随机初始化
        # 各个维度的含义是 (num_layers, minibatch_size, hidden_dim)
        '''     
        return (torch.autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                torch.autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))  
    
    def forward(self,x0,x1):
        #seq,batch,hidden
        #第一个
        _,self.hidden_0=self.lstm_0(x0.view(len(x0),self.batch_size,self.word_dim),self.hidden_0)  
            
        #第二个
        _,self.hidden_1=self.lstm_1(x1.view(len(x1),self.batch_size,self.word_dim),self.hidden_1)  

        
        #合并
        x_0=self.hidden_0[0].view(-1,self.hidden_dim)
        x_1=self.hidden_1[0].view(-1,self.hidden_dim)
        x_cat=torch.cat([x_0,x_1],dim=1)
        out=self.linear(x_cat)
        out=F.log_softmax(out,dim=1)
        return out

* loss vs model

In [ ]:
word_dim=32
hidden_dim=24
target_size=2
model = lstm_embedding_cat(word_dim, hidden_dim,target_size)
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

* train

In [ ]:

seq_len0=np.random.randint(10)+5
seq_len1=np.random.randint(10)+5
for i in range(10):   
    
    x0=torch.randn((seq_len0,word_dim))
    x1=torch.randn((seq_len1,word_dim))
    y=torch.LongTensor(torch.randint(low=0,high=2,size=(1,)).long())
    
    model.zero_grad()
    model.hidden_0=model.init_hidden()
    model.hidden_1=model.init_hidden()
    # Step 3. 前向传播
    tag_scores = model(x0,x1)
    # Step 4. 计算损失和梯度值 通过调用 optimizer.step() 来更新梯度
    loss = loss_function(tag_scores, y)
    loss.backward()
    optimizer.step()
    print(loss.item())

### 6.欠采样